In [ ]:
import time
import random
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pyspssio
import re
import networkx as nx

sns.set_palette("pastel")
sns.set_theme(style="whitegrid", palette="pastel", context="notebook")

df, meta = pyspssio.read_sav("29-я волна_репрезентативная выборка_SPSS/ДОМОХОЗЯЙСТВА/r29h_os_71.sav")

In [ ]:
import networkx as nx

# Создаем граф
DG = nx.DiGraph()
DG.add_edges_from([
    (1, 2, {'tip': 'suprug', 'now': 'da'}),
    (2, 1, {'tip': 'suprug', 'now': 'da'}),
    (1, 3, {'tip': 'blood_ditya', 'now': 'da'}),
    (1, 4, {'tip': 'priyem_ditya', 'now': 'da'}),
    (2, 3, {'tip': 'blood_ditya', 'now': 'da'}),
    (2, 4, {'tip': 'blood_ditya', 'now': 'da'}),
    (5, 1, {'tip': 'blood_ditya', 'now': 'da'}),
])

# Функция для фильтрации входящих ребер по типу связи
def has_no_incoming_blood_or_priyem_edges(graph, node):
    incoming_edges = graph.in_edges(node, data=True)
    return all(data['tip'] not in ['blood_ditya', 'priyem_ditya'] for _, _, data in incoming_edges)

# Найти все узлы, которые не имеют входящих ребер типа 'blood_ditya' или 'priyem_ditya'
possible_roots = [node for node in DG.nodes if has_no_incoming_blood_or_priyem_edges(DG, node)]

# Если несколько корней, выбираем тот, который охватывает больше всего потомков
root = max(possible_roots, key=lambda node: len(nx.descendants(DG, node)))

print(f"Найденный корень: {root}")

# Функция для вычисления максимальной глубины графа от найденного корня
def max_depth(graph, root):
    depths = nx.single_source_shortest_path_length(graph, root)
    return max(depths.values()) + 1  # +1 для включения самого корня как первого поколения

# Вычисляем количество поколений
num_generations = max_depth(DG, root)
print(f"Количество поколений в семье: {num_generations}")


In [ ]:
def has_no_outgoing_blood_or_priyem_edges(graph, node):
    outgoing_edges = graph.out_edges(node, data=True)
    return all(data['tip'] not in ['blood_ditya', 'priyem_ditya'] for _, _, data in outgoing_edges)

# Найти всех узлов, которые не имеют исходящих ребер типа 'blood_ditya' или 'priyem_ditya'
children = [node for node in DG.nodes if has_no_outgoing_blood_or_priyem_edges(DG, node)]

# Выводим количество детей
num_children = len(children)
print(f"Количество детей в семье: {num_children}")

In [ ]:
def is_priyem_ditya(graph, node):
    incoming_edges = graph.in_edges(node, data=True)
    return any(data['tip'] == 'priyem_ditya' for _, _, data in incoming_edges)


# Из этих детей отфильтровать неродных детей (тех, у кого есть входящие ребра типа 'priyem_ditya')
non_blood_children = [node for node in children if is_priyem_ditya(DG, node)]

# Выводим количество неродных детей
num_non_blood_children = len(non_blood_children)
print(f"Количество неродных детей в семье: {num_non_blood_children}")

In [ ]:
# Функция для получения родителей ребенка
def get_parents(graph, node):
    incoming_edges = graph.in_edges(node, data=True)
    parents = [parent for parent, _, data in incoming_edges if data['tip'] in ['blood_ditya', 'priyem_ditya']]
    return parents

# Проверка, является ли семья полной
def is_complete_family(graph):
    children = [node for node in graph.nodes if has_no_outgoing_blood_or_priyem_edges(graph, node)]
    for child in children:
        parents = get_parents(graph, child)
        if len(parents) < 2:  # У ребенка меньше двух родителей
            return False
    return True

complete_family = is_complete_family(DG)
print(f"Семья полная: {'да' if complete_family else 'нет'}")

In [ ]:
# Функция для фильтрации входящих ребер по типу связи
def has_no_incoming_blood_or_priyem_edges(graph, node):
    incoming_edges = graph.in_edges(node, data=True)
    return all(data['tip'] not in ['blood_ditya', 'priyem_ditya'] for _, _, data in incoming_edges)

# Найти все узлы, которые не имеют входящих ребер типа 'blood_ditya' или 'priyem_ditya'
possible_roots = [node for node in tg.nodes if has_no_incoming_blood_or_priyem_edges(tg, node)]

# Если несколько корней, выбираем тот, который охватывает больше всего потомков
root = max(possible_roots, key=lambda node: len(nx.descendants(tg, node)))

print(f"Найденный корень: {root}")

# Функция для вычисления максимальной глубины графа от найденного корня
def max_depth(graph, root):
    depths = nx.single_source_shortest_path_length(graph, root)
    return max(depths.values()) + 1  # +1 для включения самого корня как первого поколения

# Вычисляем количество поколений
num_generations = max_depth(tg, root)
print(f"Количество поколений в семье: {num_generations}")


def has_no_outgoing_blood_or_priyem_edges(graph, node):
    outgoing_edges = graph.out_edges(node, data=True)
    return all(data['tip'] not in ['blood_ditya', 'priyem_ditya'] for _, _, data in outgoing_edges)

# Найти всех узлов, которые не имеют исходящих ребер типа 'blood_ditya' или 'priyem_ditya'
children = [node for node in tg.nodes if has_no_outgoing_blood_or_priyem_edges(tg, node)]

# Выводим количество детей
num_children = len(children)
print(f"Количество детей в семье: {num_children}")


def is_priyem_ditya(graph, node):
    incoming_edges = graph.in_edges(node, data=True)
    return any(data['tip'] == 'priyem_ditya' for _, _, data in incoming_edges)


# Из этих детей отфильтровать неродных детей (тех, у кого есть входящие ребра типа 'priyem_ditya')
non_blood_children = [node for node in children if is_priyem_ditya(tg, node)]

# Выводим количество неродных детей
num_non_blood_children = len(non_blood_children)
print(f"Количество неродных детей в семье: {num_non_blood_children}")


# Функция для получения родителей ребенка
def get_parents(graph, node):
    incoming_edges = graph.in_edges(node, data=True)
    parents = [parent for parent, _, data in incoming_edges if data['tip'] in ['blood_ditya', 'priyem_ditya']]
    return parents

# Проверка, является ли семья полной
def is_complete_family(graph):
    children = [node for node in graph.nodes if has_no_outgoing_blood_or_priyem_edges(graph, node)]
    for child in children:
        parents = get_parents(graph, child)
        if len(parents) < 2:  # У ребенка меньше двух родителей
            return False
    return True

complete_family = is_complete_family(tg)
print(f"Семья полная: {'да' if complete_family else 'нет'}")